# Experimentación para desarrollar funciones API necesarias
En esta Notebook se desarrollan y prueban las funciones que son solicitadas para la API. Al desplegarlas, podrían ajustarse por requisitos de FastAPI o Render.

## Importaciones

In [1]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")


## Carga de los dataset necesarios

In [ ]:
# Cargar el archivo Parquet en un DataFrame
max_playtime_by_genre_year = pd.read_parquet('../Datasets/max_playtime_by_genre_year.parquet')
max_playtime_per_genre = pd.read_parquet('../Datasets/max_playtime_per_genre.parquet')
summary_playtime_per_year = pd.read_parquet('../Datasets/summary_playtime_per_year.parquet')
top3_recomendados = pd.read_parquet('../Datasets/top3_recomendados.parquet')
top3_no_recomendados = pd.read_parquet('../Datasets/top3_no_recomendados.parquet')
conteo_sentimientos = pd.read_parquet('../Datasets/conteo_sentimientos.parquet')
# Cargar datos para el sistema de recomendacion
#df_modelo = pd.read_parquet('../Datasets/recomendacion.parquet')
item_vectors  = pd.read_parquet('../Datasets/item_vectors.parquet')
unique_item_ids  = pd.read_parquet('../Datasets/unique_item_ids.parquet')
#user_vectors  = pd.read_parquet('../Datasets/user_vectors.parquet')

## Función PlayTimeGenre(genero:str)
La función PlayTimeGenre tiene por parametro 'genero', filtrará los datos por el género proporcionado y calculará el año con más horas jugadas para ese género.<br>
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}


In [46]:
def PlayTimeGenre(genero: str):
    # La función PlayTimeGenre tiene por parametro 'genero', filtrará los datos por el género proporcionado
    # y calculará el año con más horas jugadas para ese género..

    # Convertir el género a minúsculas para hacer la comparación sin ser sensible a mayúsculas
    genero = genero.lower()

    # Verificar si el género está presente en el DataFrame
    if genero not in max_playtime_by_genre_year['genres'].str.lower().unique():
        return {"Error": f"El género '{genero}' no se encuentra en el conjunto de datos."}

    # Filtrar el DataFrame por el género proporcionado
    genre_data = max_playtime_by_genre_year[max_playtime_by_genre_year['genres'].str.lower() == genero]

    # Obtener el año con más horas jugadas para el género dado
    max_playtime_year = genre_data['release_year'].values[0] if not genre_data.empty else None



    # Crear el diccionario para el formato JSON
    result = {f'Año de lanzamiento con más horas jugadas para {genero.capitalize()}': str(max_playtime_year)}

    return result

In [47]:
# Ejemplo de uso de la función con un género específico, por ejemplo, 'Shooter'
genre_result = PlayTimeGenre('rpg')

# Convertir el resultado a formato JSON
json_result = json.dumps(genre_result, ensure_ascii=False)
print(json_result)


{"Año de lanzamiento con más horas jugadas para Rpg": "2011"}


## Función UserForGenre(genero:str)
Esta función tiene por parametro 'genero', debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.<br>
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [48]:
def UserForGenre(genre: str):
    # Esta función tiene por parametro 'genero', debe devolver el usuario que acumula más horas jugadas 
    # para el género dado y una lista de la acumulación de horas jugadas por año.

    genre = genre.lower()  # Convertir el género a minúsculas para ser insensible a mayúsculas/minúsculas

    # Filtrar por el género deseado en el dataframe max_playtime_per_genre
    genre_data = max_playtime_per_genre[max_playtime_per_genre['genres'].str.lower() == genre]

    if len(genre_data) == 0:
        return "Género no encontrado"

    # Obtener el usuario con más horas jugadas para el género dado
    max_playtime_user = genre_data.loc[genre_data['playtime_forever'].idxmax()]['user_id']

    # Filtrar por el género deseado en el dataframe summary_playtime_per_year
    genre_year_data = summary_playtime_per_year[summary_playtime_per_year['user_id'].isin(genre_data['user_id'])]

    # Agrupar y sumar las horas jugadas por año, excluyendo los años con 0 horas jugadas
    yearly_hours = genre_year_data.groupby('release_year')['playtime_total_hours'].sum().reset_index()
    yearly_hours = yearly_hours[yearly_hours['playtime_total_hours'] > 1]

    # Crear un diccionario con el formato solicitado
    hours_by_year = [
        {'Año': int(year), 'Horas': int(hours)}
        for year, hours in zip(yearly_hours['release_year'], yearly_hours['playtime_total_hours'])
    ]

    # Formato de salida en JSON
    output = {
        f'Usuario con más horas jugadas para Género {genre.capitalize()}': max_playtime_user,
        'Horas jugadas': hours_by_year
    }

    return output

In [49]:
# Ejemplo de uso de la función
genero_deseado = 'action'  # Reemplaza 'Género X' con el género específico que desees buscar
resultado_json = UserForGenre(genero_deseado)
print(resultado_json)  # Imprimir el resultado en formato JSON

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre', 'Horas jugadas': [{'Año': 1995, 'Horas': 10}, {'Año': 1997, 'Horas': 4}, {'Año': 1999, 'Horas': 10}, {'Año': 2000, 'Horas': 1177}, {'Año': 2001, 'Horas': 4}, {'Año': 2002, 'Horas': 7}, {'Año': 2003, 'Horas': 129}, {'Año': 2004, 'Horas': 2134}, {'Año': 2005, 'Horas': 355}, {'Año': 2006, 'Horas': 3040}, {'Año': 2007, 'Horas': 1932}, {'Año': 2008, 'Horas': 149}, {'Año': 2009, 'Horas': 1902}, {'Año': 2010, 'Horas': 2153}, {'Año': 2011, 'Horas': 11560}, {'Año': 2012, 'Horas': 11190}, {'Año': 2013, 'Horas': 7924}, {'Año': 2014, 'Horas': 5404}, {'Año': 2015, 'Horas': 26683}, {'Año': 2016, 'Horas': 3073}, {'Año': 2017, 'Horas': 3605}]}


## Función UsersRecommend(anio:int)
Esta función tiene por parametro 'anio', Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales).<br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}].

In [50]:
def UsersRecommend(anio):
    # Verificar si el año proporcionado tiene datos en el DataFrame top3_recomendados
    if anio not in top3_recomendados['year'].unique():
        return f"No hay datos disponibles para el año {anio}"

    # Filtrar el DataFrame top3_recomendados para el año proporcionado
    top_games_year = top3_recomendados[top3_recomendados['year'] == anio]

    # Verificar si hay menos de 3 juegos para el año consultado
    if len(top_games_year) < 1:
        return "No hay suficientes juegos para mostrar el top 3"

    # Reiniciar el índice del DataFrame filtrado por año
    top_games_year = top_games_year.reset_index(drop=True)

    # Obtener el top 3 de juegos más recomendados para el año dado
    top_3_games = top_games_year.head(3)

    # Crear una lista con el formato especificado
    top_3_list = []
    for index, row in top_3_games.iterrows():
        game_rank = index + 1  # Usar el índice + 1 como número de puesto
        game_dict = {"Puesto " + str(game_rank): row['title']}
        top_3_list.append(game_dict)

    return top_3_list

In [51]:
# Ejemplo de uso: Obtener el top 3 de juegos recomendados para el año 2022
anio_elegido = 2015
resultados = UsersRecommend(anio_elegido)
resultados

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Unturned'},
 {'Puesto 3': 'Team Fortress 2'}]

## Función UsersNotRecommend(anio:int)
Esta función tiene por parametro 'anio', Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos).<br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}].

In [52]:
def UsersNotRecommend(anio):
    # Verificar si el año proporcionado tiene datos en el DataFrame top3_no_recomendados
    if anio not in top3_no_recomendados['year'].unique():
        return f"No hay datos disponibles para el año {anio}"

    # Filtrar el DataFrame top3_no_recomendados para el año proporcionado
    top_games_year = top3_no_recomendados[top3_no_recomendados['year'] == anio]

    # Verificar si hay menos de 3 juegos para el año consultado
    if len(top_games_year) < 1:
        return "No hay suficientes juegos para mostrar el top 3 menos recomendados"

    # Reiniciar el índice del DataFrame filtrado por año
    top_games_year = top_games_year.reset_index(drop=True)

    # Obtener el top 3 de juegos más recomendados para el año dado
    top_3_games = top_games_year.head(3)

    # Crear una lista con el formato especificado
    top_3_list = []
    for index, row in top_3_games.iterrows():
        game_rank = index + 1  # Usar el índice + 1 como número de puesto
        game_dict = {"Puesto " + str(game_rank): row['title']}
        top_3_list.append(game_dict)

    return top_3_list

In [53]:
# Ejemplo de uso: Obtener el top 3 de juegos recomendados para el año 2022
anio_elegido = 2015
resultados = UsersNotRecommend(anio_elegido)
resultados

[{'Puesto 1': 'Act of Aggression - Reboot Edition'},
 {'Puesto 2': 'Age of Empires II HD'},
 {'Puesto 3': 'Alien Swarm'}]

## Función sentiment_analysis(anio:int)
Esta función tiene por parametro 'anio', Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.<br>
Ejemplo de retorno: {Negativo = 182, Neutral = 120, Positivo = 278}

In [54]:
def sentiment_analysis(año):
    # Filtrar el DataFrame por el año dado
    df_filtrado = conteo_sentimientos[conteo_sentimientos['release_year'] == año]
    
    if df_filtrado.empty:
        return f"No hay datos disponibles para el año {año}"
    
    # Obtener la suma de cada sentimiento para el año dado
    sentimientos_totales = df_filtrado[['Negativo', 'Neutral', 'Positivo']].sum()
    
    # Convertir el resultado en un diccionario con valores enteros
    resultado = sentimientos_totales.astype(int).to_dict()
    
    return resultado

In [55]:

# Ejemplo de uso:
año = 2015  # Cambia este valor al año que desees analizar
resultados = sentiment_analysis(año)
print(resultados)


{'Negativo': 1207, 'Neutral': 8432, 'Positivo': 3234}


## Función recomendacion_juego( id de producto )
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [4]:
def recomendacion_juego(input_game):
    if input_game not in unique_item_ids['item_id'].unique():
        return f"No hay datos disponibles para el Id: {input_game}"

    # Encontrar el vector de ítem correspondiente para el juego de entrada
    input_game_vector = item_vectors.loc[input_game].values.reshape(1, -1)
    
    # Calcular la similitud del coseno entre el vector del juego de entrada y todos los demás vectores de ítems
    similarities = cosine_similarity(input_game_vector, item_vectors.values)
    
    # Ordenar los puntajes de similitud del coseno en orden descendente
    similar_games_indices = similarities.argsort()[0][::-1]
    similar_games_indices = similar_games_indices[1:]  # Excluir el primer índice (el juego de entrada)
    
    # Seleccionar los mejores N juegos con los puntajes de similitud del coseno más altos como juegos recomendados
    recommended_game_ids = item_vectors.index[similar_games_indices][:5]
    
    # Obtener los nombres de los juegos recomendados, considerando solo un registro por 'item_id'
    recommended_games = unique_item_ids[unique_item_ids['item_id'].isin(recommended_game_ids)]['item_name'].tolist()
    
    return recommended_games


In [5]:
# Test1 recomendacion_juego 
input_game = 209650
recommended_games = recomendacion_juego(input_game)
print(recommended_games)


['PAYDAY 2', 'Watch_Dogs', 'Call of Duty: Black Ops III', 'Quake Live', 'Resident Evil™: Operation Raccoon City']


In [8]:
# Test2 recomendacion_juego 
input_game = 303210
recommended_games = recomendacion_juego(input_game)
print(recommended_games)


['The Elder Scrolls V: Skyrim', 'Loadout', 'Carpe Diem', 'Batman™: Arkham Knight', 'Rocksmith 2014']


## Función recomendacion_usuario( id de usuario )

Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [59]:
def recomendacion_usuario(input_user):
    input_user = str(input_user)

    vector_usuario = user_vectors.loc[input_user].values.reshape(1, -1)
    similarities = cosine_similarity(vector_usuario, user_vectors.values)

    similar_users_indices = similarities.argsort()[0][::-1]
    top_similar_users = user_vectors.index[similar_users_indices][:5]

    recommended_games = []
    for user in top_similar_users:
        games = df_modelo.loc[df_modelo['user_id'] == user, 'item_id'].tolist()
        recommended_games.extend(games)

    recommended_games = list(set(recommended_games))
    recommended_games.sort(key=lambda x: user_vectors.loc[top_similar_users, x].mean(), reverse=True)
    recommended_games = recommended_games[:5]

    #juegos_recomendados_nombres = [id_name_map[item_id] for item_id in recommended_games]
    juegos_recomendados_nombres = unique_item_ids[unique_item_ids['item_id'].isin(recommended_games)]['item_name'].tolist()
    
    return juegos_recomendados_nombres

In [60]:
# Test1 a la función
usuario = 'Scalarius'

recomendaciones = recomendacion_usuario(usuario)
print(f"Las recomendaciones para el usuario {usuario} son: {recomendaciones}")


Las recomendaciones para el usuario Scalarius son: ['Fallout 4', 'Call of Duty: Modern Warfare 2', 'Goat Simulator', 'Portal 2', 'Reflex']


In [61]:
# Test2 a la función
usuario = 'Coop2303'

recomendaciones = recomendacion_usuario(usuario)
print(f"Las recomendaciones para el usuario {usuario} son: {recomendaciones}")


Las recomendaciones para el usuario Coop2303 son: ['Left 4 Dead 2', 'Double Action: Boogaloo', 'Portal 2', 'Velvet Sundown', 'Distance']
